In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [10]:
conditions_df = pd.read_csv("competition_dataset/env_parameter_sample.csv")
conditions_df

,time,TAN_DF,pH_DF,NO2_DF,Alkalinity_DF,TAN_DGS,pH_DGS,NO2_DGS,Alkalinity_DGS,Flow_rate,Treatment,Module,TAN_removal_biocarrier,co2_mgl,h2s_ugl,o2_mgl,o2_sat,salinity,temp,sample_name
0,29/08/2022 08:30,0.240,7.880,0.64,342.40,0.180,7.860,0.71,344.00,60.00,200,12,0.000034,7.03,0.16,7.79,83.78,14.88,15.1,B1_1
1,29/08/2022 08:31,0.280,8.070,0.64,433.40,0.220,8.020,0.55,433.90,60.00,200,1,0.000038,4.93,0.32,8.41,89.33,13.02,15.1,A1_1
2,02/09/2022 08:30,0.220,7.650,0.51,219.80,0.150,8.031,0.76,211.00,60.00,200,12,0.000040,7.99,0.17,7.82,85.07,16.38,15.1,No_sample
3,02/09/2022 08:31,0.200,7.800,0.51,236.00,0.070,8.030,0.53,236.30,60.00,200,1,0.000083,6.76,0.26,8.21,87.59,13.40,15.1,No_sample
4,05/09/2022 08:30,0.160,7.600,0.46,234.00,0.120,7.900,0.62,234.00,60.00,200,12,0.000023,8.76,0.13,7.57,81.98,16.36,15.3,B1_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,11/01/2023 08:31,0.255,7.128,0.06,72.29,0.135,7.411,0.05,73.07,54.55,70,1,0.000081,8.33,0.26,8.60,96.42,14.82,15.1,No_sample
120,12/01/2023 08:30,0.220,7.553,0.08,184.10,0.130,7.771,0.06,185.80,62.75,200,12,0.000070,7.66,0.13,8.28,94.29,17.28,15.3,No_sample
121,12/01/2023 08:31,0.225,7.226,0.08,74.13,0.125,7.298,0.05,72.83,57.25,70,1,0.000071,7.89,0.23,8.56,95.37,14.73,15.0,No_sample
122,16/01/2023 08:30,0.250,7.564,0.08,192.60,0.165,7.688,0.07,205.90,73.65,200,12,0.000078,7.49,0.18,8.24,93.87,17.21,15.2,B9_3


Some rows do not have a sample id assigned, even though most of them have just been taken one minute after one that does, so we'll interpolate with that the mssing data.

Note that the values are already ordered by date, so we'll work with that also.

In [11]:
for j,row in conditions_df.iterrows():
    
    if j == 0:
        continue
    
    else:
        if row["sample_name"] == "No_sample":
            if row["time"][:15] == conditions_df.iloc[j-1]["time"][:15]:
                if int(row["time"][-1]) == int(conditions_df.iloc[j-1]["time"][-1]) + 1:
                    conditions_df.loc[j,"sample_name"] = conditions_df.iloc[j-1]["sample_name"]
        


In [12]:
conditions_df

,time,TAN_DF,pH_DF,NO2_DF,Alkalinity_DF,TAN_DGS,pH_DGS,NO2_DGS,Alkalinity_DGS,Flow_rate,Treatment,Module,TAN_removal_biocarrier,co2_mgl,h2s_ugl,o2_mgl,o2_sat,salinity,temp,sample_name
0,29/08/2022 08:30,0.240,7.880,0.64,342.40,0.180,7.860,0.71,344.00,60.00,200,12,0.000034,7.03,0.16,7.79,83.78,14.88,15.1,B1_1
1,29/08/2022 08:31,0.280,8.070,0.64,433.40,0.220,8.020,0.55,433.90,60.00,200,1,0.000038,4.93,0.32,8.41,89.33,13.02,15.1,A1_1
2,02/09/2022 08:30,0.220,7.650,0.51,219.80,0.150,8.031,0.76,211.00,60.00,200,12,0.000040,7.99,0.17,7.82,85.07,16.38,15.1,No_sample
3,02/09/2022 08:31,0.200,7.800,0.51,236.00,0.070,8.030,0.53,236.30,60.00,200,1,0.000083,6.76,0.26,8.21,87.59,13.40,15.1,No_sample
4,05/09/2022 08:30,0.160,7.600,0.46,234.00,0.120,7.900,0.62,234.00,60.00,200,12,0.000023,8.76,0.13,7.57,81.98,16.36,15.3,B1_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,11/01/2023 08:31,0.255,7.128,0.06,72.29,0.135,7.411,0.05,73.07,54.55,70,1,0.000081,8.33,0.26,8.60,96.42,14.82,15.1,No_sample
120,12/01/2023 08:30,0.220,7.553,0.08,184.10,0.130,7.771,0.06,185.80,62.75,200,12,0.000070,7.66,0.13,8.28,94.29,17.28,15.3,No_sample
121,12/01/2023 08:31,0.225,7.226,0.08,74.13,0.125,7.298,0.05,72.83,57.25,70,1,0.000071,7.89,0.23,8.56,95.37,14.73,15.0,No_sample
122,16/01/2023 08:30,0.250,7.564,0.08,192.60,0.165,7.688,0.07,205.90,73.65,200,12,0.000078,7.49,0.18,8.24,93.87,17.21,15.2,B9_3


Filter rows that have a sample code:

In [13]:
conditions_df = conditions_df[conditions_df["sample_name"] != "No_sample"]

Add sample code to data so its  more easy to filter

In [14]:
for j,row in conditions_df.iterrows():
    conditions_df.loc[j,"System"] = row["sample_name"][0]
    conditions_df.loc[j,"Treatment"] = row["sample_name"][1]
    conditions_df.loc[j,"TimeStamp"] = row["sample_name"][3:]

conditions_df

/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/ipykernel_67480/1767432388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conditions_df.loc[j,"System"] = row["sample_name"][0]
/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/ipykernel_67480/1767432388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conditions_df.loc[j,"TimeStamp"] = row["sample_name"][3:]


,time,TAN_DF,pH_DF,NO2_DF,Alkalinity_DF,TAN_DGS,pH_DGS,NO2_DGS,Alkalinity_DGS,Flow_rate,...,TAN_removal_biocarrier,co2_mgl,h2s_ugl,o2_mgl,o2_sat,salinity,temp,sample_name,System,TimeStamp
0,29/08/2022 08:30,0.240,7.880,0.640,342.40,0.180,7.860,0.710,344.00,60.000000,...,0.000034,7.03,0.16,7.79,83.78,14.88,15.10,B1_1,B,1
1,29/08/2022 08:31,0.280,8.070,0.640,433.40,0.220,8.020,0.550,433.90,60.000000,...,0.000038,4.93,0.32,8.41,89.33,13.02,15.10,A1_1,A,1
4,05/09/2022 08:30,0.160,7.600,0.460,234.00,0.120,7.900,0.620,234.00,60.000000,...,0.000023,8.76,0.13,7.57,81.98,16.36,15.30,B1_2,B,2
5,05/09/2022 08:31,0.150,7.700,0.460,233.40,0.040,8.000,0.350,232.00,60.000000,...,0.000070,7.61,0.20,8.20,86.99,13.89,15.00,A1_2,A,2
12,12/09/2022 08:30,0.160,7.565,0.270,204.40,0.090,7.828,0.220,218.30,60.000000,...,0.000050,9.08,0.11,7.40,82.05,16.98,15.20,B1_3,B,3
13,12/09/2022 08:31,0.120,7.650,0.180,196.20,0.060,7.930,0.150,199.20,60.000000,...,0.000038,6.82,0.20,7.96,86.20,14.23,14.80,A1_3,A,3
18,19/09/2022 08:30,0.290,7.150,0.340,88.72,0.200,7.460,0.290,87.96,54.000000,...,0.000064,11.73,0.21,7.27,79.89,16.96,15.20,B2_1,B,1
19,19/09/2022 08:31,0.250,7.314,0.240,96.43,0.140,7.611,0.210,96.34,49.000000,...,0.000067,9.38,0.27,8.10,87.01,14.31,14.90,A2_1,A,1
26,26/09/2022 08:30,0.460,7.108,0.410,88.44,0.100,7.426,0.110,86.31,44.000000,...,0.000196,13.55,0.26,7.30,82.76,17.04,15.20,B2_2,B,2
27,26/09/2022 08:31,0.180,7.358,0.300,88.50,0.170,7.596,0.130,88.21,66.000000,...,0.000008,8.42,0.21,8.13,90.40,14.29,15.15,A2_2,A,2


In [15]:
conditions_df.to_csv("conditions_df.csv",index=False)

Note that there are treatments specified in the sample name and then treatment specified in the water, they don't seem to be related at all as in the conditions data there are three different and in the sample code 9.

# How does treatment affect water conditions?

Our approach is going to be to attempt to create a DL classifiers that given the input water conditions over time is going to output what treatment was applied.

Once we manage to have a model that very accuratelly predicts this we'll start adding noise to certain variables to systematically identify which ones are related to the treatment, which might mean which ones are affected by the treatment in an specific way, meaning that other treatments do differently.

For start all columns in:

Ok, start with knn:

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

cols = conditions_df.columns[1:-3]
X = conditions_df[cols]
y = conditions_df["Treatment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
print(f"KNN score: {neigh.score(X_test,y_test)}")

KNN score: 0.4


SVM:

In [18]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)
print(f"SVM score: {clf.score(X_test,y_test)}")

SVM score: 0.1


Proably there is no effect of treatment on water conditions.

No luck here, dead end.

# What conditions are related to the increase or decrease of certain species?

In [19]:
species_df = pd.read_csv("sample_by_specie_timewise.csv")
species_df.head()

,id,Specie,System,Treatment,Replica,t1,t2,t3,slope,var,std,mean
0,Homo sapiens_A1_A,Homo sapiens,A,1,A,0.262329,0.327225,0.831947,0.284809,0.064824,0.254606,0.473834
1,Homo sapiens_A1_B,Homo sapiens,A,1,B,0.318878,0.447284,0.941915,0.311519,0.072147,0.268602,0.569359
2,Homo sapiens_A1_C,Homo sapiens,A,1,C,0.286041,0.518135,0.730994,0.222476,0.033018,0.181708,0.511723
3,Homo sapiens_A2_A,Homo sapiens,A,2,A,0.641026,0.587084,0.000000,-0.320513,0.084277,0.290305,0.409370
4,Homo sapiens_A2_B,Homo sapiens,A,2,B,0.729927,0.543478,0.000000,-0.364964,0.095881,0.309646,0.424468


In [20]:
#filter where t1+t2+t3 is non zero
filtered_df = species_df[species_df['t1'] + species_df['t2'] + species_df['t3'] > 0]
filtered_df

,id,Specie,System,Treatment,Replica,t1,t2,t3,slope,var,std,mean
0,Homo sapiens_A1_A,Homo sapiens,A,1,A,0.262329,0.327225,0.831947,0.284809,0.064824,0.254606,0.473834
1,Homo sapiens_A1_B,Homo sapiens,A,1,B,0.318878,0.447284,0.941915,0.311519,0.072147,0.268602,0.569359
2,Homo sapiens_A1_C,Homo sapiens,A,1,C,0.286041,0.518135,0.730994,0.222476,0.033018,0.181708,0.511723
3,Homo sapiens_A2_A,Homo sapiens,A,2,A,0.641026,0.587084,0.000000,-0.320513,0.084277,0.290305,0.409370
4,Homo sapiens_A2_B,Homo sapiens,A,2,B,0.729927,0.543478,0.000000,-0.364964,0.095881,0.309646,0.424468
...,...,...,...,...,...,...,...,...,...,...,...,...
628629,Youhaiella tibetensis_B1_B,Youhaiella tibetensis,B,1,B,0.056465,0.000000,0.000000,-0.028233,0.000709,0.026618,0.018822
628630,Youhaiella tibetensis_B1_C,Youhaiella tibetensis,B,1,C,0.046642,0.000000,0.000000,-0.023321,0.000483,0.021987,0.015547
628764,Zhongshania aliphaticivorans_B9_A,Zhongshania aliphaticivorans,B,9,A,0.155521,0.000000,0.000000,-0.077760,0.005375,0.073313,0.051840
628765,Zhongshania aliphaticivorans_B9_B,Zhongshania aliphaticivorans,B,9,B,0.156006,0.000000,0.000000,-0.078003,0.005408,0.073542,0.052002


In [21]:
cols = conditions_df.columns[1:-3].delete(list(conditions_df.columns[1:-3]).index("Treatment"))
conditions = {}
for j,row in conditions_df.iterrows():
    conditions[row["System"] + row["Treatment"]] = row[cols].tolist()

In [22]:
print(cols)

Index(['TAN_DF', 'pH_DF', 'NO2_DF', 'Alkalinity_DF', 'TAN_DGS', 'pH_DGS',
       'NO2_DGS', 'Alkalinity_DGS', 'Flow_rate', 'Module',
       'TAN_removal_biocarrier', 'co2_mgl', 'h2s_ugl', 'o2_mgl', 'o2_sat',
       'salinity', 'temp'],
      dtype='object')


# Trying an lstm for regression of the species values

In [23]:
import math

In [24]:
class Dataset():
    def __init__(self, species_df, conditions):
        self.species_df = species_df
        self.conditions = conditions

        #diferent species
        self.species_vocab = species_df["Specie"].unique().tolist()

    
    def __len__(self):
        return len(self.species_df)*3 #because 3 timesteps
    
    def __getitem__(self, idx):
        assert ((idx >= 0) & (idx < self.__len__())), "Dataset index out of range"

        real_id = int(math.ceil((idx+1)/3))
        t = 2 - (real_id*3 - (idx+1) ) +1
        
        real_id -= 1 #fix
        
        #specie one hot
        spc_name = self.species_df.iloc[real_id]["Specie"]
        one_hot_idx = self.species_vocab.index(spc_name)
        y = self.species_df.iloc[real_id]["t"+str(t)]
        
        return  torch.tensor([one_hot_idx],dtype=int) , torch.tensor([t] + conditions[str(self.species_df.iloc[real_id]["System"]) + str(self.species_df.iloc[real_id]["Treatment"]) ],dtype=torch.float) , torch.tensor(y,dtype=torch.float)

In [25]:
#wrapper dataset
class train_dataset():
    def __init__(self,dataset):
        self.dataset = dataset
        self.lenght = len(dataset) - int(len(dataset)/4)

    def __len__(self):
        return self.lenght

    def __getitem__(self,idx):
        assert ((idx >= 0) & (idx < self.lenght)), "Dataset index out of range"
        return self.dataset.__getitem__(idx)

class test_dataset():
    def __init__(self,dataset):
        self.dataset = dataset
        self.lenght = int(len(dataset)/4)
        self.offset = int(len(dataset)/4)*3
    
    def __len__(self):
        return self.lenght

    def __getitem__(self,idx):
        assert ((idx >= 0) & (idx < self.lenght)), "Dataset index out of range"
        return self.dataset.__getitem__(idx+self.offset)


In [26]:
dataset = Dataset(filtered_df,conditions)

In [27]:
train_ds = train_dataset(dataset)
test_ds = test_dataset(dataset)

In [28]:
batch_size = 100
train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
test_dl = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last=True)

# Simple FCNN

Define model:

In [5]:
class FCNN_model(nn.Module):
    def __init__(self, input_dim, embedding_dim, data_dim, hidden_dim, output_dim):
        super(FCNN_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(num_embeddings=input_dim,embedding_dim=embedding_dim) 
        self.fc = nn.Linear(data_dim,hidden_dim)
        self.rel = nn.ReLU()
        self.fc_1 = nn.Linear(hidden_dim,hidden_dim) 
        self.fc_regr = nn.Linear(hidden_dim,output_dim) 
        
        
    def forward(self, onehot, x):
        #ic(x.shape)
        emb = self.embedding(onehot)
        data_enc = self.fc(x)
        data_enc = self.rel(data_enc)
        
        out = emb + data_enc
        out = self.fc_1(out)
        out = self.rel(out)
        out = self.fc_regr(out)
        out = self.rel(out)

        return out

In [34]:
input_dim = len(dataset.species_vocab)
data_dim = len(cols) + 1
embedding_dim = 1024
hidden_dim = 1024
output_dim = 1

In [35]:
import torch
from torchviz import make_dot

# Assuming you have already defined your FCNN_model
model = FCNN_model(input_dim, embedding_dim, data_dim, hidden_dim, output_dim)

# Dummy input data for visualization (replace with actual data if needed)
dummy_onehot = torch.zeros((batch_size, input_dim),dtype=int)
dummy_x = torch.zeros((batch_size, data_dim))

# Forward pass to generate the computational graph
output = model(dummy_onehot, dummy_x)
dot = make_dot(output, params=dict(model.named_parameters()))

# Save the visualization as a PDF or display it in Jupyter Notebook
dot.render("fcnn_model_graph")  # Saves as "fcnn_model_graph.pdf"
# Or display it in Jupyter Notebook
dot.view()

RuntimeError: The size of tensor a (1360) must match the size of tensor b (100) at non-singleton dimension 1

: 

: 

In [22]:
class FCNN_model(nn.Module):
    def __init__(self, input_dim, embedding_dim, data_dim, hidden_dim, output_dim):
        super(FCNN_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(num_embeddings=input_dim,embedding_dim=embedding_dim) 
        self.fc = nn.Linear(data_dim,hidden_dim)
        self.rel = nn.ReLU()
        self.fc_1 = nn.Linear(hidden_dim,hidden_dim) 
        self.fc_regr = nn.Linear(hidden_dim,output_dim) 
        
        
    def forward(self, onehot, x):
        #ic(x.shape)
        emb = self.embedding(onehot)
        data_enc = self.fc(x)
        data_enc = self.rel(data_enc)
        
        out = emb + data_enc
        out = self.fc_1(out)
        out = self.rel(out)
        out = self.fc_regr(out)
        out = self.rel(out)


        return out

In [23]:
class AttentionBlock(nn.Module):
    def __init__(self, key_dim, val_dim, query_dim, hidden_dim, num_heads,FC_hidden=500):
        super(AttentionBlock, self).__init__()
        self.key_gen = nn.Sequential(
            nn.Linear(key_dim,FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden,FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden, hidden_dim),
            nn.ReLU(),
        )

        self.val_gen = nn.Sequential(
            nn.Linear(val_dim, FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden, FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden, hidden_dim),
            nn.ReLU(),
        )
        
        self.query_gen = nn.Sequential(
            nn.Linear(query_dim, FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden, FC_hidden),
            nn.ReLU(),
            nn.Linear(FC_hidden, hidden_dim),
            nn.ReLU(),
        )
        self.multihead_attn = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
    
    def forward(self,keys,values,queries):
        key = self.key_gen(keys) #generate key with FC key is directly the embedding
        value = self.val_gen(values) #generate value with FC
        query = self.query_gen(queries) #generate query with FC
        output, _ =  self.multihead_attn(key=key, value=value, query=query)
        return output
        

In [24]:
from icecream import ic
class Attentive_model(nn.Module):
    def __init__(self, input_dim, embedding_dim, data_dim, hidden_dim, output_dim):
        super(Attentive_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(num_embeddings=input_dim,embedding_dim=embedding_dim) 
        self.selfattn1 =  AttentionBlock(key_dim=data_dim + embedding_dim, val_dim=data_dim + embedding_dim, query_dim=data_dim + embedding_dim, hidden_dim=hidden_dim, num_heads=1)
        self.rel = nn.ReLU()
        self.fc_1 = nn.Linear(hidden_dim,hidden_dim) 
        self.fc_regr = nn.Linear(hidden_dim,output_dim) 
        
        
    def forward(self, onehot, x):
        #ic(x.shape)
        emb = self.embedding(onehot)
        inp = torch.unsqueeze(x,dim=1)
        inp = torch.cat((emb,inp),dim=-1)
        out = self.selfattn1(inp,inp,inp)
        out = self.fc_1(out)
        out = self.rel(out)
        out = self.fc_regr(out)
        out = self.rel(out)

        return out

In [25]:
device = torch.device("cpu")

In [7]:
input_dim = len(dataset.species_vocab)
data_dim = len(cols) + 1
embedding_dim = 500
hidden_dim = 1024
output_dim = 1

model = Attentive_model(input_dim, embedding_dim, data_dim,hidden_dim, output_dim).to(device)

print(model)

NameError: name 'dataset' is not defined

In [27]:
import wandb
!wandb login

wandb: Currently logged in as: haradai. Use `wandb login --relogin` to force relogin


In [28]:
wandb.init(project='datahon_uib_2023', name='embedding+Attention')

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 100, in run
    sreq = self._sock_client.read_server_request()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 274, in read_server_request
    data = self._read_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 248, in _read_packet_bytes
    rec = self._extract_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 230, in _extract_packet_bytes
    assert magic == ord("W")
AssertionError
wandb: Currently logged in as: haradai. Use `wandb login --relogin` to force relogi

In [32]:
def train(dataloader,test_dataloader, model, batch_size, num_epochs):
    model.train()

    criterion =  nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    
    for epoch in range(num_epochs):
        losses = []
        for batch, (onehot, condits, y) in enumerate(dataloader):
            
            #x = x.to(device)
            #y = y.to(device)
            optimizer.zero_grad()
            
            y_pred = model(onehot,condits)
            #ic(y.shape)
            #ic(y_pred.shape)
            loss = criterion(y_pred,torch.reshape(y,(batch_size,1,1)))  #cross entropy loss needs (N,C,seq_lenght)

            loss.backward(retain_graph=True)
            optimizer.step()

            if batch%30 == 0:
                print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
                losses.append(loss.item())
            
            wandb.log({'train_loss': loss.item()})
        
        wandb.log({'epoch_train_loss': np.mean(losses)})

        #test 
        model.eval()  
        test_losses = []
        with torch.no_grad(): 
            for batch, (onehot, condits, y) in enumerate(test_dataloader):
                y_pred = model(onehot,condits)
                loss = criterion(y_pred,y)
                if batch%30 == 0:
                    print({ 'TEST!': epoch, 'batch': batch, 'loss': loss.item() })
                test_losses.append(loss)
            wandb.log({'epoch_test_loss': np.mean(test_losses)})
    
    return losses

In [33]:
train(train_dl,test_dl,model,batch_size=batch_size,num_epochs=20)

{'epoch': 0, 'batch': 0, 'loss': 0.09215878695249557}
{'epoch': 0, 'batch': 30, 'loss': 0.12118127197027206}
{'epoch': 0, 'batch': 60, 'loss': 0.07644344121217728}
{'epoch': 0, 'batch': 90, 'loss': 0.08196799457073212}
{'epoch': 0, 'batch': 120, 'loss': 1.1371899843215942}
{'epoch': 0, 'batch': 150, 'loss': 0.4136393070220947}
{'epoch': 0, 'batch': 180, 'loss': 0.017749806866049767}
{'epoch': 0, 'batch': 210, 'loss': 0.0645543783903122}
{'epoch': 0, 'batch': 240, 'loss': 0.0985645204782486}
{'epoch': 0, 'batch': 270, 'loss': 0.030047528445720673}
{'epoch': 0, 'batch': 300, 'loss': 0.07810259610414505}
{'epoch': 0, 'batch': 330, 'loss': 0.3852981626987457}
{'epoch': 0, 'batch': 360, 'loss': 0.12967230379581451}
{'epoch': 0, 'batch': 390, 'loss': 0.032143834978342056}
{'epoch': 0, 'batch': 420, 'loss': 0.12927885353565216}


Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 100, in run
    sreq = self._sock_client.read_server_request()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 274, in read_server_request
    data = self._read_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 248, in _read_packet_bytes
    rec = self._extract_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 230, in _extract_packet_bytes
    assert magic == ord("W")
AssertionError


{'epoch': 0, 'batch': 450, 'loss': 0.07661717385053635}
{'epoch': 0, 'batch': 480, 'loss': 0.15576985478401184}
{'epoch': 0, 'batch': 510, 'loss': 0.06309325248003006}
{'epoch': 0, 'batch': 540, 'loss': 0.040909409523010254}
{'epoch': 0, 'batch': 570, 'loss': 0.6278598308563232}
{'epoch': 0, 'batch': 600, 'loss': 0.08705823868513107}


/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


{'TEST!': 0, 'batch': 0, 'loss': 0.02642090991139412}
{'TEST!': 0, 'batch': 30, 'loss': 0.038871973752975464}
{'TEST!': 0, 'batch': 60, 'loss': 0.09206195920705795}
{'TEST!': 0, 'batch': 90, 'loss': 0.04129450023174286}
{'TEST!': 0, 'batch': 120, 'loss': 0.3476561903953552}
{'TEST!': 0, 'batch': 150, 'loss': 0.3718233108520508}
{'TEST!': 0, 'batch': 180, 'loss': 0.10150834172964096}
{'epoch': 1, 'batch': 0, 'loss': 0.4378369450569153}
{'epoch': 1, 'batch': 30, 'loss': 0.06263614445924759}
{'epoch': 1, 'batch': 60, 'loss': 0.10188274085521698}
{'epoch': 1, 'batch': 90, 'loss': 0.5720087885856628}
{'epoch': 1, 'batch': 120, 'loss': 0.021893322467803955}
{'epoch': 1, 'batch': 150, 'loss': 0.3111087381839752}
{'epoch': 1, 'batch': 180, 'loss': 0.03828584402799606}
{'epoch': 1, 'batch': 210, 'loss': 0.12831175327301025}
{'epoch': 1, 'batch': 240, 'loss': 0.08868546783924103}
{'epoch': 1, 'batch': 270, 'loss': 0.31197795271873474}
{'epoch': 1, 'batch': 300, 'loss': 0.036563776433467865}
{'ep

...

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 100, in run
    sreq = self._sock_client.read_server_request()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 274, in read_server_request
    data = self._read_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 248, in _read_packet_bytes
    rec = self._extract_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 230, in _extract_packet_bytes
    assert magic == ord("W")
AssertionError


{'epoch': 19, 'batch': 30, 'loss': 0.8895406126976013}
{'epoch': 19, 'batch': 60, 'loss': 1.302069902420044}
{'epoch': 19, 'batch': 90, 'loss': 0.09492199867963791}
{'epoch': 19, 'batch': 120, 'loss': 0.2643193006515503}
{'epoch': 19, 'batch': 150, 'loss': 0.1299688220024109}
{'epoch': 19, 'batch': 180, 'loss': 0.06820698827505112}
{'epoch': 19, 'batch': 210, 'loss': 0.06518743187189102}
{'epoch': 19, 'batch': 240, 'loss': 0.026898954063653946}
{'epoch': 19, 'batch': 270, 'loss': 13.00701904296875}
{'epoch': 19, 'batch': 300, 'loss': 0.11312364786863327}
{'epoch': 19, 'batch': 330, 'loss': 0.07124859094619751}
{'epoch': 19, 'batch': 360, 'loss': 0.0977214053273201}
{'epoch': 19, 'batch': 390, 'loss': 0.07513831555843353}
{'epoch': 19, 'batch': 420, 'loss': 0.08351410180330276}
{'epoch': 19, 'batch': 450, 'loss': 0.014249681495130062}
{'epoch': 19, 'batch': 480, 'loss': 0.7396978735923767}
{'epoch': 19, 'batch': 510, 'loss': 0.43333736062049866}
{'epoch': 19, 'batch': 540, 'loss': 0.332

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 100, in run
    sreq = self._sock_client.read_server_request()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 274, in read_server_request
    data = self._read_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 248, in _read_packet_bytes
    rec = self._extract_packet_bytes()
  File "/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 230, in _extract_packet_bytes
    assert magic == ord("W")
AssertionError


{'TEST!': 19, 'batch': 30, 'loss': 0.030268078669905663}
{'TEST!': 19, 'batch': 60, 'loss': 0.12702533602714539}
{'TEST!': 19, 'batch': 90, 'loss': 0.05199022218585014}
{'TEST!': 19, 'batch': 120, 'loss': 0.11401596665382385}
{'TEST!': 19, 'batch': 150, 'loss': 0.037202585488557816}
{'TEST!': 19, 'batch': 180, 'loss': 0.12057416886091232}


[0.34136688709259033,
 0.8895406126976013,
 1.302069902420044,
 0.09492199867963791,
 0.2643193006515503,
 0.1299688220024109,
 0.06820698827505112,
 0.06518743187189102,
 0.026898954063653946,
 13.00701904296875,
 0.11312364786863327,
 0.07124859094619751,
 0.0977214053273201,
 0.07513831555843353,
 0.08351410180330276,
 0.014249681495130062,
 0.7396978735923767,
 0.43333736062049866,
 0.3325459957122803,
 0.26808086037635803,
 0.06947805732488632]